<a href="https://colab.research.google.com/github/swapnilmn/Assignment_03/blob/main/Question_1_Ass_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question 1 (15 Marks)
Build a RNN based seq2seq model which contains the following layers: (i) input layer for character embeddings (ii) one encoder RNN which sequentially encodes the input character sequence (Latin) (iii) one decoder RNN which takes the last state of the encoder as input and produces one output character at a time (Devanagari). 


#Librares

In [ ]:
import csv
import numpy as np
import random
import torch
import torch.nn as nn 
import torch.nn.functional as F
import time
import matplotlib.pyplot as plt
import math

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

!pip install wandb -qU
import wandb
wandb.login()
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.8 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Mounted at /content/drive


# Load of data

In [ ]:
import math  # Import the math module for mathematical operations
import time  # Import the time module for time-related functions
import torch  # Import the torch module for working with tensors
import numpy as np  # Import the numpy module for numerical operations

def asMinutes(s):
    # Convert seconds to minutes and seconds
    m = math.floor(s / 60)  # Calculate the number of minutes
    s -= m * 60  # Subtract the minutes converted to seconds from the total seconds
    return '%dm %ds' % (m, s)  # Return the formatted string for minutes and seconds

def span(since, percent):
    # Calculate the elapsed time and estimated remaining time
    now = time.time()  # Get the current time
    s = now - since  # Calculate the elapsed time in seconds
    es = s / (percent)  # Calculate the estimated total time in seconds
    rs = es - s  # Calculate the remaining time in seconds
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))  # Return the formatted string for elapsed time and remaining time

def key_presenting(dictionary, values):
    # Find keys in the dictionary that match the given values
    found_keys = []  # Initialize an empty list to store the found keys
    for value in values:  # Iterate over the given values
        for key, val in dictionary.items():  # Iterate over the dictionary items
            if value == val:  # If the value matches the dictionary value
                found_keys.append(key)  # Add the key to the list of found keys
    
    if found_keys:  # If any keys are found
        str = ''.join(found_keys)  # Concatenate the found keys into a single string
        if str[-1] == '\n':  # If the string ends with a newline character
            str = str[:-1]  # Remove the newline character from the end of the string
        elif str[0] == '\t':  # If the string starts with a tab character
            str = str[1:]  # Remove the tab character from the start of the string
    else:  # If no keys are found
        print("No keys found for the given values.")  # Print a message indicating no keys were found
    
    return str  # Return the string containing the found keys

def char_acc(targets, outputs):
    # Calculate character-level accuracy
    with torch.no_grad():  # Disable gradient calculation for efficiency
        count = 0  # Initialize a counter for correct predictions
        total_count = 0  # Initialize a counter for total predictions
        for i in range(targets.shape[0]):  # Iterate over the rows of the targets tensor
            same_elements = []  # Initialize an empty list to store the comparison results
            for j in range(targets.shape[1]):  # Iterate over the columns of the targets tensor
                if targets[i][j] != 67 or targets[i][j] != 66:  # Check if the target value is not equal to 67 or 66
                    same_elements.append(outputs[i][j].item() == targets[i][j].item())  # Append the comparison result to the list
            count += np.sum(same_elements)  # Add the number of True values in the list to the count
            total_count += len(same_elements)  # Add the total number of elements in the list to the total count
    return count / total_count  # Return the character-level accuracy as a ratio of correct predictions to total predictions

def word_acc(targets, outputs):
    # Calculate word-level accuracy
    outputs1 = torch.argmax(outputs, dim=1)  # Get the index of the maximum value along the second dimension of the outputs tensor
    with torch.no_grad():  # Disable gradient calculation for efficiency
        count = 0  # Initialize a counter for correct predictions
        for i in range(targets.shape[0]):  # Iterate over the rows of the targets tensor
            if (outputs1[i] == targets[i]).sum().item() == targets.shape[1]:  # Check if all elements in the row are equal
                count = count + 1  # Increment the count if all elements are equal
    return count / targets.shape[0]  # Return the word-level accuracy as a ratio of correct predictions to total predictions

def equi_points(data, epochs):
    # Sample equidistant points from data
    step = len(data) // epochs  # Calculate the step size to sample equidistant points
    indices = np.arange(0, len(data), step)  # Generate indices using numpy arange function
    equidistant_points = [data[i] for i in indices]  # Extract the equidistant points from the data list
    
    return equidistant_points  # Return the list of equidistant points


In [ ]:
import csv

def read_csv_files():
    # Define the file paths for the CSV files
    file_paths = {
        'file1': '/content/drive/MyDrive/aksharantar_sampled/mar/mar_test.csv',  # Path to 'mar_test.csv'
        'file2': '/content/drive/MyDrive/aksharantar_sampled/mar/mar_valid.csv',  # Path to 'mar_valid.csv'
        'file3': '/content/drive/MyDrive/aksharantar_sampled/mar/mar_train.csv'  # Path to 'mar_train.csv'
    }

    data = {}  # Dictionary to store the CSV data

    for file_key, file_path in file_paths.items():
        with open(file_path) as file:
            csv_reader = csv.reader(file)
            header = next(csv_reader)  # Get the header row of the CSV
            rows = [row for row in csv_reader]  # Get all the data rows of the CSV
            data[file_key] = {
                'header': header,
                'rows': rows
            }

    return data

# Execute the function to read CSV files
csv_data = read_csv_files()

# Access the data
test = csv_data['file1']['rows']  # Contains rows from 'mar_test.csv'
val = csv_data['file2']['rows']  # Contains rows from 'mar_valid.csv'
train = csv_data['file3']['rows']  # Contains rows from 'mar_train.csv'


# Data Preprocessing

In [ ]:
# Define a function to read data from a list of pairs
def Data_Reading(list):
  i = []  # List to store the first element of each pair
  t = []  # List to store the second element of each pair
  for pair in list:
    i.append(pair[0])  # Append the first element to 'i'
    t.append(pair[1])  # Append the second element to 't'
  return i, t  # Return the two lists

# Read data from the 'train' list using the 'Data_Reading' function
train_inputs, train_targets = Data_Reading(train)

# Read data from the 'test' list using the 'Data_Reading' function
test_inputs, test_targets = Data_Reading(test)

# Read data from the 'val' list using the 'Data_Reading' function
val_inputs, val_targets = Data_Reading(val)

# Print the second element of the 'train_inputs' list
# print(train_inputs[1])

# Print the second element of the 'train_targets' list
# print(train_targets[1])

# Define a dictionary to store variable values
variable_dict = {
    'sc': '\t',    # 'sc' represents a tab character
    'eb': '\n',    # 'eb' represents a newline character
    'bc ': ' ',    # 'bc' represents a space character
    'unc': '\r'    # 'unc' represents a carriage return character
}

# Accessing the values from the dictionary
sc = variable_dict['sc']    # Assign the value of '\t' to 'sc'
eb = variable_dict['eb']    # Assign the value of '\n' to 'eb'
bc  = variable_dict['bc ']  # Assign the value of ' ' to 'bc'
unc = variable_dict['unc']  # Assign the value of '\r' to 'unc'


# Function to create dictionaries for language encoding

def dictlang(inputs, targets):
    dict = {}  # Dictionary for input language
    mil = 0  # Maximum input language length
    ichar = []  # List of characters in input language

    dict_t = {}  # Dictionary for target language
    mtl = 0  # Maximum target language length
    tchar = []  # List of characters in target language

    for s in inputs:
        mil = max(len(s), mil)
        for char in s:
            if char not in dict:
                dict[char] = len(ichar)
                ichar.append(char)

    # Add special characters if they are not present in the input language dictionary
    if bc not in dict:
        dict[bc] = len(ichar)
        ichar.append(bc)

    dict[unc] = len(ichar)
    ichar.append(unc)

    # Add special characters if they are not present in the target language dictionary
    if sc not in dict_t:
        dict_t[sc] = len(tchar)
        tchar.append(sc)

    for s in targets:
        mtl = max(len(s) + 2, mtl)
        for char in s:
            if char not in dict_t:
                dict_t[char] = len(tchar)
                tchar.append(char)

    # Add special characters if they are not present in the target language dictionary
    if eb not in dict_t:
        dict_t[eb] = len(tchar)
        tchar.append(eb)

    # Add special characters if they are not present in the target language dictionary
    if bc not in dict_t:
        dict_t[bc] = len(tchar)
        tchar.append(bc)

    return dict, mil, ichar, dict_t, mtl, tchar

# Create dictionaries for language encoding using training, validation, and test inputs and targets
dict, mil, ichar, dict_t, mtl, tchar = dictlang(train_inputs + val_inputs + test_inputs, train_targets + val_targets + test_targets)


def encoding_w(a, d, ml, l):
    encs = []  # List to store encoded words
    for word in a:
        enc = []  # List to store encoded characters of a word
        for char in word:
            if char in d:
                enc.append(d[char])  # Encode character if present in the dictionary
            else:
                enc.append(d[unc])  # Encode unknown character
        if (l == 0):
            while len(enc) < ml:
                enc.append(d[bc])  # Pad with blank character if required
        if (l == 1):
            enc.insert(0, d[sc])  # Insert start character at the beginning
            while len(enc) < ml - 1:
                enc.append(d[bc])  # Pad with blank character if required
            enc.append(d[eb])  # Append end character at the end
        encs.append(enc)  # Add encoded word to the list
    return encs

def tokenize(train,val,test,dict,dict_t,mil,mtl):
    # Tokenize the data for training, validation, and testing sets
    ti, tt = Data_Reading(train)  # Read training data
    tti, ttt = Data_Reading(test)  # Read testing data
    vi, vt = Data_Reading(val)  # Read validation data

    # Encode the training, validation, and testing data
    e_t_i = encoding_w(ti,dict,mil,0)  # Encode training input data
    e_t_t = encoding_w(tt,dict_t,mtl,1)  # Encode training target data
    e_v_i = encoding_w(vi,dict,mil,0)  # Encode validation input data
    e_v_t = encoding_w(vt,dict_t,mtl,1)  # Encode validation target data
    e_tt_i = encoding_w(tti,dict,mil,0)  # Encode testing input data
    e_tt_t = encoding_w(ttt,dict_t,mtl,1)  # Encode testing target data

    return e_t_i, e_t_t, e_v_i, e_v_t, e_tt_i, e_tt_t

# Tokenize the data using the given parameters
e_t_i, e_t_t, e_v_i, e_v_t, e_tt_i, e_tt_t = tokenize(train, val, test, dict, dict_t, mil, mtl)

# Generate a random number within the range [0, 100]
r = random.randint(0, 100)

# Print the keys corresponding to the values of the encoded data at index r
# print(key_presenting(dict, e_t_i[int(r)]))
# print(key_presenting(dict_t, e_t_t[int(r)]))


# Define a function to convert input and target data into tensor pairs
def tensor_conversion(ti, tg):
    pr = []
    for id, td in zip(ti, tg):
        # Convert each item in ti to a torch tensor
        it = torch.tensor(id)
        # Convert each item in tg to a torch tensor
        tt = torch.tensor(td)
        # Append the tensor pair to the result list
        pr.append((it, tt))
    return pr

# Convert e_t_i and e_t_t to tensor pairs
e_t_p = tensor_conversion(e_t_i, e_t_t)

# Convert e_v_i and e_v_t to tensor pairs
e_v_p = tensor_conversion(e_v_i, e_v_t)

# Re-assign e_t_p with new tensor pairs converted from e_tt_i and e_tt_t
e_t_p = tensor_conversion(e_tt_i, e_tt_t)

# Create a tuple containing e_t_p, e_v_p, and e_t_p
pairs = (e_t_p, e_v_p, e_t_p)

# Randomly choose a pair from e_t_p
pair = random.choice(e_t_p)

# Print the keys in dict corresponding to the values in pair[0]
# print(key_presenting(dict, pair[0]))

# Print the keys in dict_t corresponding to the values in pair[1]
# print(key_presenting(dict_t, pair[1]))


In [ ]:
class EncoderRNN(nn.Module):
    def __init__(this, device, cell_type, vocab_size, embed_dim, hidden_size, num_layers=1, bidirectional=False, dropout_p=0):
        super(EncoderRNN, this).__init__()
        this.embedding = nn.Embedding(vocab_size, embed_dim)  # Embedding layer for converting input indices to dense vectors.
        this.hidden_size = hidden_size  # Dimensionality of the hidden state of the RNN.
        this.num_layers = num_layers  # Number of recurrent layers.
        this.bidirectional = bidirectional  # Flag indicating whether the RNN is bidirectional.
        this.cell_type = cell_type  # Type of the RNN cell: 'lstm', 'rnn', or 'gru'.
        this.dropout_p = dropout_p  # Dropout probability.
        this.dropout = nn.Dropout(this.dropout_p)  # Dropout layer for regularization.
        if cell_type == 'lstm':
            this.rnn = nn.LSTM(embed_dim, hidden_size, num_layers=this.num_layers, batch_first=True, dropout=dropout_p, bidirectional=this.bidirectional)  # LSTM RNN module.
        elif cell_type == 'rnn':
            this.rnn = nn.RNN(embed_dim, hidden_size, num_layers=this.num_layers, batch_first=True, dropout=dropout_p, bidirectional=this.bidirectional)  # Vanilla RNN module.
        elif cell_type == 'gru':
            this.rnn = nn.GRU(embed_dim, hidden_size, num_layers=this.num_layers, batch_first=True, dropout=dropout_p, bidirectional=this.bidirectional)  # GRU module.

    def forward(this, x, hidden, cell):
        out = this.embedding(x)  # Convert input indices to dense vectors.
        out = this.dropout(out)  # Apply dropout to the input.
        if this.cell_type == 'lstm':
            out, (hidden, cell) = this.rnn(out, (hidden, cell))  # Forward pass through the LSTM RNN.
            return out, hidden, cell  # Return the output, hidden state, and cell state.
        elif this.cell_type == 'rnn':
            out, hidden = this.rnn(out, hidden)  # Forward pass through the vanilla RNN.
            return out, hidden  # Return the output and hidden state.
        elif this.cell_type == 'gru':
            out, hidden = this.rnn(out, hidden)  # Forward pass through the GRU.
            return out, hidden  # Return the output and hidden state.

    def init_hidden(this, batch_size):
        hidden = torch.randn((1 + int(this.bidirectional)) * this.num_layers, batch_size, this.hidden_size, device=device)  # Initialize the hidden state tensor.
        cell = torch.randn((1 + int(this.bidirectional)) * this.num_layers, batch_size, this.hidden_size, device=device)  # Initialize the cell state tensor.
        return hidden, cell


In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, device, cell_type, output_vocab, embed_size, hidden_size, max_length, dropout_p=0.1, num_layers=1, bidirectional=False):
        super(DecoderRNN, self).__init__()
        # Initialize the decoder attributes
        self.hidden_size = hidden_size  # Size of the hidden state
        self.output_size = output_vocab  # Size of the output vocabulary
        self.embed_size = embed_size  # Size of the embedding
        self.dropout_p = dropout_p  # Dropout probability
        self.cell_type = cell_type  # Type of the RNN cell
        self.max_length = max_length  # Maximum length of the input sequence
        self.device = device  # Device to be used (e.g., 'cpu' or 'cuda')
        self.num_layers = num_layers  # Number of layers in the RNN
        self.embedding_decoder = nn.Embedding(self.output_size, self.embed_size)  # Embedding layer
        self.dropout = nn.Dropout(self.dropout_p)  # Dropout layer
        self.bidirectional = bidirectional  # Flag indicating bidirectional RNN

        # Initialize the RNN based on the specified cell_type
        if cell_type == 'lstm':
            self.rnn = nn.LSTM(self.embed_size, hidden_size, num_layers=self.num_layers, batch_first=True, bidirectional=self.bidirectional, dropout=self.dropout_p)
        elif cell_type == 'gru':
            self.rnn = nn.GRU(self.embed_size, hidden_size, num_layers=self.num_layers, batch_first=True, bidirectional=self.bidirectional, dropout=self.dropout_p)
        elif cell_type == 'rnn':
            self.rnn = nn.RNN(self.embed_size, hidden_size, num_layers=self.num_layers, batch_first=True, bidirectional=self.bidirectional, dropout=self.dropout_p)

        self.out = nn.Linear((1 + int(self.bidirectional)) * self.hidden_size, self.output_size)  # Linear layer
        self.out_activation = nn.LogSoftmax(dim=-1)  # Log softmax activation function

    def forward(self, input, hidden, cell):
        input = input.unsqueeze(1)  # Add a dimension of size 1 to the input tensor
        embedded_decoder = self.embedding_decoder(input)  # Embed the input tensor
        embedded_decoder = self.dropout(embedded_decoder)  # Apply dropout to the embedded tensor

        # Pass the embedded tensor through the RNN
        if self.cell_type == 'lstm':
            output, (hidden, cell) = self.rnn(embedded_decoder, (hidden, cell))
        elif self.cell_type == 'gru':
            output, hidden = self.rnn(embedded_decoder, hidden)
        elif self.cell_type == 'rnn':
            output, hidden = self.rnn(embedded_decoder, hidden)

        output = F.relu(self.out(output))  # Apply ReLU activation to the output tensor
        output = F.log_softmax(output, dim=-1)  # Apply log softmax activation to the output tensor

        return output, hidden, cell

    def init_hidden(self, encoder_hidden, encoder_cell, encoder_bidirectional):
        hidden = encoder_hidden[-(1 + int(encoder_bidirectional)):].repeat(self.num_layers, 1, 1)  # Repeat the hidden tensor
        cell = encoder_cell[-(1 + int(encoder_bidirectional)):].repeat(self.num_layers, 1, 1)  # Repeat the cell tensor
        return hidden, cell


In [ ]:
class Seq2Seq(nn.Module):
    def __init__(this, encoder, decoder, device):
        super().__init__()
        this.encoder = encoder
        this.decoder = decoder
        this.device = device
        this.mtl = 0  # Initialize the maximum target length
        this.sos = 0  # Initialize the start of sequence token
        
    def forward(this, source, target, teacher_forcing_ratio = 0.5):
        # Get dimensions of input
        batch_size = target.shape[0]
        target_len = target.shape[1]
        this.mtl = target_len  # Set the maximum target length
        target_vocab_size = this.decoder.output_size
        
        # Initialize tensor for storing outputs
        outputs = torch.zeros(batch_size, target_len, target_vocab_size).to(this.device)

        # Initialize hidden states of the encoder
        encoder_hidden, encoder_cell = this.encoder.init_hidden(batch_size)

        # Forward pass through the encoder
        if (this.encoder.cell_type == 'lstm'):
            encoder_outputs, encoder_hidden, encoder_cell = this.encoder.forward(source, encoder_hidden, encoder_cell)
        if (this.encoder.cell_type == 'rnn'):
            encoder_outputs, encoder_hidden = this.encoder.forward(source, encoder_hidden, encoder_cell)
        if (this.encoder.cell_type == 'gru'):
            encoder_outputs, encoder_hidden = this.encoder.forward(source, encoder_hidden, encoder_cell)

        # First input to the decoder is the <sos> tokens
        input = target[:,0]
        this.sos = target[:,0]

        # Initialize hidden states of the decoder
        hidden, cell = this.decoder.init_hidden(encoder_hidden, encoder_cell, this.encoder.bidirectional)
        
        # Iterate over the target sequence
        for t in range(1, target_len):
            # Forward pass through the decoder
            output, hidden, cell = this.decoder.forward(input, hidden, cell)
            outputs[:,t] = output.squeeze(1)  # Store the decoder output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(-1)
            input = target[:,t] if teacher_force else top1.squeeze(1)

        return outputs

    def inference(this, source, target):
        # Get dimensions of input
        batch_size = source.shape[0]
        target_len = this.mtl
        target_vocab_size = this.decoder.output_size
        outputs = torch.zeros(batch_size, target_len, target_vocab_size).to(this.device)

        # Initialize hidden states of the encoder
        encoder_hidden, encoder_cell = this.encoder.init_hidden(batch_size)

        # Forward pass through the encoder
        if (this.encoder.cell_type == 'lstm'):
            encoder_outputs, encoder_hidden, encoder_cell = this.encoder.forward(source, encoder_hidden, encoder_cell)
        if (this.encoder.cell_type == 'rnn'):
            encoder_outputs, encoder_hidden = this.encoder.forward(source, encoder_hidden, encoder_cell)
        if (this.encoder.cell_type == 'gru'):
            encoder_outputs, encoder_hidden = this.encoder.forward(source, encoder_hidden, encoder_cell)
           
        # First input to the decoder is the <sos> token
        input = this.sos

        # Initialize hidden states of the decoder
        hidden, cell = this.decoder.init_hidden(encoder_hidden, encoder_cell, this.encoder.bidirectional)
      
        # Iterate over the target sequence
        for t in range(1, target_len):
            # Forward pass through the decoder
            output, hidden, cell = this.decoder.forward(input, hidden, cell)
            outputs[:,t] = output.squeeze(1)  # Store the decoder output
            top1 = output.argmax(-1)
            input = top1.squeeze(1)

        # print(f"Outputs = {outputs[:,0]}")
        # print(f"Targets = {target[:,0]}")
        
        return outputs


In [ ]:
# Commented code

# Define the input and output dimensions based on the length of the dictionary
input_dim = len(dict)
output_dim = len(dict_t)

# Set the batch size for training and validation
batch_size = 32
val_batch_size = 32

# Set the embedding dimensions for the encoder and decoder
enc_embedding = 256
dec_embedding = 256

# Set the number of hidden units in the encoder and decoder
hidden = 512

# Set the number of layers in the encoder and decoder
enc_num_layers = 3
dec_num_layers = 2

# Set the dropout probabilities for the encoder and decoder
enc_dropout = 0.4
dec_dropout = 0.4

# Set the maximum length for the input sequence
max_length = mtl

# Set the cell type for the encoder and decoder
cell_type = 'gru'

# Initialize the encoder, decoder, and the overall model
enc = EncoderRNN(device, cell_type, input_dim, enc_embedding, hidden, enc_num_layers, bidirectional=True, dropout_p=enc_dropout)
dec = DecoderRNN(device, cell_type, output_dim, dec_embedding, hidden, max_length, dec_dropout, dec_num_layers, bidirectional=True)
model = Seq2Seq(enc, dec, device).to(device)

# Function to count the number of trainable parameters in the model
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Print the number of trainable parameters in the model
print(f'The model has {count_parameters(model):,} trainable parameters')

# Define the optimizer and the loss criterion
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.NLLLoss()

# Function to train the model for a given number of iterations
def trainIters(model, pairs, batch_size, n_iters, optimizer, tf, print_every=10, plot_every=10, log=True, Attention=False):
    start = time.time()
    plot_losses = []
    train_char_accuracy = []
    train_word_accuracy = []
    val_losses = []
    val_char_accuracy = []
    val_word_accuracy = []
    print_loss_total = 0
    plot_loss_total = 0
    print_val_loss_total = 0
    plot_val_loss_total = 0

    train_pairs = pairs[0]
    val_pairs = pairs[1]
    train_accuracy = 0

    criterion = nn.NLLLoss()

    count = 0
    for iter in range(1, n_iters + 1):
        for i in np.arange(start=0, stop=len(train_pairs) - batch_size, step=batch_size):
            train_accuracy = 0
            count += 1
            if (i + batch_size > len(train_pairs)):
                batch_size = len(train_pairs) - i + 1
            input_tensor = []
            target_tensor = []

            for j in range(batch_size):
                input_tensor.append(train_pairs[i + j][0])
                target_tensor.append(train_pairs[i + j][1])

            input_tensor = torch.stack(input_tensor).squeeze(1).long().cuda()
            target_tensor = torch.stack(target_tensor).squeeze(1).long().cuda()

            optimizer.zero_grad()
            if (count < 4000):
                out = model(input_tensor, target_tensor, teacher_forcing_ratio=tf)
            else:
                out = model(input_tensor, target_tensor, teacher_forcing_ratio=0)

            out = torch.permute(out, [0, 2, 1])
            loss = criterion(out, target_tensor)

            train_accuracy_word = word_acc(target_tensor, out) * batch_size
            train_accuracy = train_accuracy + train_accuracy_word
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
            optimizer.step()

            print_loss_total += loss
            plot_loss_total += loss

            if count % 800 == 0:
                val_input_tensor = []
                val_target_tensor = []

                for j in range(batch_size):
                    val_input_tensor.append(val_pairs[j][0])
                    val_target_tensor.append(val_pairs[j][1])

                val_input_tensor = torch.stack(val_input_tensor).squeeze(1).long().cuda()
                val_target_tensor = torch.stack(val_target_tensor).squeeze(1).long().cuda()
                if (Attention == True):
                    val_out, _ = model.inference(val_input_tensor, val_target_tensor)
                    val_out = val_out.permute(0, 2, 1)
                else:
                    val_out = model.inference(val_input_tensor, val_target_tensor)
                    val_out = val_out.permute(0, 2, 1)
                val_loss = criterion(val_out, val_target_tensor)
                val_loss_sampled = val_loss
                wandb.log({'Val_Loss': val_loss_sampled})

            if count % 800 == 0:
                print_loss_avg = print_loss_total / 800
                print_loss_total = 0
                print('%s (%d %d%%) %.7f' % (span(start, iter / n_iters),
                                             iter, iter / n_iters * 100, print_loss_avg))

            if count % 800 == 0:
                plot_loss_avg = plot_loss_total / 800
                plot_losses.append(plot_loss_avg.detach())
                wandb.log({'Train Loss': plot_loss_avg})

                plot_loss_total = 0

        train_accuracy = train_accuracy / (len(train_pairs) - batch_size)
        train_word_accuracy.append(train_accuracy)

    print(train_word_accuracy)
    plot_losses = [losses.cpu().numpy() for losses in plot_losses]

    char_count = 0
    word_count = 0

    for i in np.arange(start=0, stop=len(val_pairs) - batch_size, step=batch_size):
        if (i + batch_size > len(val_pairs)):
            batch_size = len(val_pairs) - i + 1
        val_input_tensor = []
        val_target_tensor = []
        for j in range(batch_size):
            val_input_tensor.append(val_pairs[i + j][0])
            val_target_tensor.append(val_pairs[i + j][1])

        val_input_tensor = torch.stack(val_input_tensor).squeeze(1).long().cuda()
        val_target_tensor = torch.stack(val_target_tensor).squeeze(1).long().cuda()
        if (Attention == True):
            val_out, _ = model.inference(val_input_tensor, val_target_tensor)
            val_out = val_out.permute(0, 2, 1)
        else:
            val_out = model.inference(val_input_tensor, val_target_tensor)
            val_out = val_out.permute(0, 2, 1)
        val_loss = criterion(val_out, val_target_tensor)

        val_accuracy_word = word_acc(val_target_tensor, val_out)
        word_count = word_count + (val_accuracy_word) * batch_size

    word_accuracy = word_count / (len(val_pairs) - batch_size)

    metrics = {'Val_Accuracy': word_accuracy}
    wandb.log(metrics)

    print(f"Val loss = {val_loss}")
    print(f'Word-level-accuracy on val set = {word_accuracy}')


The model has 18,998,083 trainable parameters


In [ ]:
# Hyperparameter configuration
sweep_config = {
    'method': 'bayes',
    'name': 'Hyperparameter Tuning-Bayesian'
}

# Metric to optimize
metric = {
    'name': 'Val_Accuracy',
    'goal': 'maximize'
}

# Assign metric to sweep config
sweep_config['metric'] = metric

# Define parameter values
parameters_dict = {
    'optimiser': {
        'values': ['nadam']
    },
    'teacher_forcing_ratio': {
        'values': [0.5, 0.7]
    },
    'bidirectional': {
        'values': [True]
    },
    'enc_embedding': {
        'values': [128]
    },
    'dec_embedding': {
        'values': [128]
    },
    'epochs': {
        'values': [2]
    },
    'hidden_size': {
        'values': [64, 128, 256]
    },
    'enc_layers': {
        'values': [3]
    },
    'dec_layers': {
        'values': [3]
    },
    'dropout': {
        'values': [0.3]
    },
    'cell_type': {
        'values': ['rnn', 'gru']
    }
}

# Assign parameters to sweep config
sweep_config['parameters'] = parameters_dict

# Initialize the sweep
sweep_id = wandb.sweep(sweep_config, project="CS6910 Assignment 3")


def train_sweep(config=None):
    with wandb.init(config=config) as run:
        config = wandb.config

        # Set up model parameters
        input_dim = len(dict)
        output_dim = len(dict_t)
        batch_size = 32
        val_batch_size = 32
        enc_embedding = config.enc_embedding
        dec_embedding = config.dec_embedding
        hidden = config.hidden_size
        enc_num_layers = config.enc_layers
        dec_num_layers = config.dec_layers
        enc_dropout = config.dropout
        dec_dropout = config.dropout
        max_length = mtl
        cell_type = config.cell_type

        # Initialize the encoder, decoder, and model
        enc = EncoderRNN(device, cell_type, input_dim, enc_embedding, hidden, enc_num_layers, bidirectional=config.bidirectional, dropout_p=enc_dropout)
        dec = DecoderRNN(device, cell_type, output_dim, dec_embedding, hidden, max_length, dec_dropout, dec_num_layers, bidirectional=config.bidirectional)
        model = Seq2Seq(enc, dec, device).to(device)

        # Set experiment name
        exp_name = str(config.cell_type)
        exp_name = exp_name
        exp_name = exp_name+'_optim_'+ config.optimiser
        wandb.run.name = exp_name

        # Set optimizer based on config
        if (config.optimiser == 'adam'):
            optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        elif (config.optimiser == 'nadam'):
            optimizer = torch.optim.NAdam(model.parameters(), lr=0.001)

        # Set loss criterion
        criterion = nn.NLLLoss()

        # Train the model
        trainIters(model, pairs, 32, config.epochs, optimizer, config.teacher_forcing_ratio)


# Run the sweep
wandb.agent(sweep_id, train_sweep, count=1)
wandb.finish()


Create sweep with ID: 6inmyl1l
Sweep URL: https://wandb.ai/ed22s009/CS6910%20Assignment%203/sweeps/6inmyl1l


wandb: Agent Starting Run: tklzn5py with config:
wandb: 	bidirectional: True
wandb: 	cell_type: rnn
wandb: 	dec_embedding: 128
wandb: 	dec_layers: 3
wandb: 	dropout: 0.3
wandb: 	enc_embedding: 128
wandb: 	enc_layers: 3
wandb: 	epochs: 2
wandb: 	hidden_size: 64
wandb: 	optimiser: nadam
wandb: 	teacher_forcing_ratio: 0.5
wandb: Currently logged in as: ed22s009. Use `wandb login --relogin` to force relogin


[0.0, 0.0]
Val loss = 1.4987664222717285
Word-level-accuracy on val set = 0.0


Val_Accuracy,▁
Val_Accuracy,0.0
